In [40]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy as astro
from astropy.modeling import models, fitting
%matplotlib qt5

### Load and display all rotational diagram data from CASSIS

In [60]:
#rotdiag_data1 = pd.read_csv("CH2(OH)CHO_1.csv", delimiter="\t")
#rotdiag_data2 = pd.read_csv("CH2(OH)CHO_2.csv", delimiter="\t")
rotdiag_data = pd.read_csv("CH3CN_1.csv", delimiter="\t")
print(rotdiag_data)
rotdiag_data = rotdiag_data.drop_duplicates(["Eup (K)"])

#res = [rotdiag_data1, rotdiag_data2]
#rotdiag_data = pd.concat(res)

rotdiag_data = rotdiag_data.reset_index()
molecule_name = "$H2CCO$"

energies = rotdiag_data["Eup (K)"]
energies = [round(x,4) for x in energies]
lognugu = rotdiag_data["Ln(Nu/gu)"]
errors = rotdiag_data["err(Ln(Nu/gu))"]

plt.errorbar(energies, lognugu, yerr=errors, fmt="o", capsize=3, color="black")
plt.ylabel("$\ln(N_u/g_u)$", fontsize=15)
plt.xlabel("$E_u/k$", fontsize=15)
plt.show()

     Eup (K)  Ln(Nu/gu)  err(Ln(Nu/gu))
0  15.975260  20.757541        0.191061
1   8.829497  21.238673        0.147000
2  41.825290  19.255699        0.224375
3  20.389921  20.600043        0.145126
4  13.244174  20.653350        0.132488
5  82.842295  19.102951        0.208411
6  25.687393  19.651254        0.152359
7  18.541808  19.516598        0.158448


### Load CASSIS line analysis file

In [61]:
line_file = pd.read_csv("CH3CN.rotd.txt", delimiter="\t", header=1, skiprows=[2])
line_file

,id,NumCompo,Species,QuantumNumbers,Frequency,Eup,Gup,Aij,FitFreq,DeltaFitFreq,...,FWHM,IntensityMax,Flux1stMom,deltaFlux1stMom,rms,deltaV,Cal,Size,TelescopePath,TelescopeName
0,41001,1,CH3CN,(4 1 _ 3 1),73588.799,15.9753,18,0.000030,73588.843,0.0673,...,1.0037,0.0441,0.0443,0.0,6.8913,0.7957,0.0,0.0,/Users/munozcar/Research/CASSIS/delivery/teles...,iram_EMIR
1,41001,1,CH3CN,(4 0 _ 3 0),73590.218,8.8295,18,0.000032,73590.260,0.0335,...,1.6958,0.0534,0.0905,0.0,6.4857,0.7957,0.0,0.0,/Users/munozcar/Research/CASSIS/delivery/teles...,iram_EMIR
2,41001,1,CH3CN,(5 2 _ 4 2),91979.994,41.8253,22,0.000053,91979.840,0.0781,...,0.6490,0.0075,0.0049,0.0,2.0861,0.6366,0.0,0.0,/Users/munozcar/Research/CASSIS/delivery/teles...,iram_EMIR
3,41001,1,CH3CN,(5 1 _ 4 1),91985.314,20.3899,22,0.000061,91985.325,0.0346,...,1.1916,0.0490,0.0584,0.0,5.5094,0.6366,0.0,0.0,/Users/munozcar/Research/CASSIS/delivery/teles...,iram_EMIR
4,41001,1,CH3CN,(5 0 _ 4 0),91987.088,13.2442,22,0.000063,91987.092,0.0241,...,1.2782,0.0548,0.0701,0.0,5.3112,0.6366,0.0,0.0,/Users/munozcar/Research/CASSIS/delivery/teles...,iram_EMIR
5,41001,1,CH3CN,(6 3 _ 5 3),110364.354,82.8423,52,0.000083,110364.221,0.0909,...,1.2899,0.0208,0.0268,0.0,4.4757,0.5306,0.0,0.0,/Users/munozcar/Research/CASSIS/delivery/teles...,iram_EMIR
6,41001,1,CH3CN,(6 1 _ 5 1),110381.372,25.6874,26,0.000108,110381.377,0.0569,...,0.7954,0.0290,0.0231,0.0,3.7550,0.5305,0.0,0.0,/Users/munozcar/Research/CASSIS/delivery/teles...,iram_EMIR
7,41001,1,CH3CN,(6 0 _ 5 0),110383.500,18.5418,26,0.000111,110383.551,0.0355,...,0.9744,0.0347,0.0338,0.0,4.2190,0.5305,0.0,0.0,/Users/munozcar/Research/CASSIS/delivery/teles...,iram_EMIR
8,41001,1,CH3CN,(6 1 _ 5 1),110381.372,25.6874,26,0.000108,110381.373,0.0464,...,0.7954,0.0290,0.0231,0.0,3.7524,0.5305,0.0,0.0,/Users/munozcar/Research/CASSIS/delivery/teles...,iram_EMIR


### Filter out detections with peak intensity < 5$\sigma$ and integrated flux < 3$\sigma$

In [62]:
rms = [float(x)*0.001 if float(x)*0.001 < 0.005 else 0.005 for x in line_file["rms"] ]# Convert to from mK to K
Imax = [float(x) for x in line_file["IntensityMax"]]
FitFlux = [float(x) for x in line_file["FitFlux"]]
Freqs = [float(x) for x in line_file["Frequency"]]
Eup = [float(x) for x in line_file["Eup"]]
Aij = [float(x) for x in line_file["Aij"]]

detections = []

for i,line in enumerate(rms):
    if Imax[i] > 5*rms[i] and FitFlux[i] > 3*rms[i]:
        detections.append([Imax[i], FitFlux[i], rms[i], Freqs[i], Eup[i], Aij[i]])
        

#detections_low = []
#detections_high = []

#for i,line in enumerate(rms):
#    if Imax[i] > 5*rms[i] and FitFlux[i] > 3*rms[i] and Eup[i]<50:
#        detections_low.append([Imax[i], FitFlux[i], rms[i], Freqs[i], Eup[i], Aij[i]])
#    elif Imax[i] > 5*rms[i] and FitFlux[i] > 3*rms[i] and Eup[i]>=50:
#        detections_high.append([Imax[i], FitFlux[i], rms[i], Freqs[i], Eup[i], Aij[i]])


#### For single component

In [63]:
filtered_energ = []
filtered_lognugu = []
filtered_err = []

for i,line in enumerate(detections):
    for j,eup in enumerate(energies):
        if eup == line[4]:
            print(line[4])
            filtered_energ.append(eup)
            filtered_lognugu.append(lognugu[j])
            filtered_err.append(errors[j])


15.9753
8.8295
20.3899
13.2442
25.6874
18.5418
25.6874


#### For multiple component

In [21]:
filtered_energ_low = []
filtered_lognugu_low = []
filtered_err_low = []

for i,line in enumerate(detections_low):
    for j,eup in enumerate(energies):
        if eup == line[4]:
            print(line[4])
            filtered_energ_low.append(eup)
            filtered_lognugu_low.append(lognugu[j])
            filtered_err_low.append(errors[j])
            
filtered_energ_high = []
filtered_lognugu_high = []
filtered_err_high = []

for i,line in enumerate(detections_high):
    for j,eup in enumerate(energies):
        if eup == line[4]:
            print(line[4])
            filtered_energ_high.append(eup)
            filtered_lognugu_high.append(lognugu[j])
            filtered_err_high.append(errors[j])



In [64]:
plt.errorbar(energies, lognugu, yerr=errors, fmt="o", capsize=3, color="gray")
plt.errorbar(filtered_energ, filtered_lognugu, yerr=filtered_err, fmt="o", capsize=3, color="red")

#plt.errorbar(filtered_energ_low, filtered_lognugu_low, yerr=filtered_err_low, fmt="o", capsize=3, color="blue")
#plt.errorbar(filtered_energ_high, filtered_lognugu_high, yerr=filtered_err_high, fmt="o", capsize=3, color="red")

plt.ylabel("$\ln(N_u/g_u)$", fontsize=15)
plt.xlabel("$E_u/k$", fontsize=15)
plt.show()

### Use filtered detections to do a least-squares fit for excitation temperature $T_{ex}$ and $\ln(N_u/g_u)$
### (One component)

In [65]:
from lmfit import minimize, Parameters, fit_report


def residual(params, x, data, eps_data):
    m = -1/params['T']
    b = params['lnNugu']
  
    model = m*x+b

    return (data-model) / eps_data


params = Parameters()
params.add('T', value=20, min=0, max=1000)
params.add('lnNugu', value=20)


out = minimize(residual, params, args=(np.array(filtered_energ), filtered_lognugu, filtered_err))

print(fit_report(out))
# <end examples/doc_fitting_withreport.py>

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 16
    # data points      = 7
    # variables        = 2
    chi-square         = 36.9948412
    reduced chi-square = 7.39896825
    Akaike info crit   = 15.6540783
    Bayesian info crit = 15.5458986
[[Variables]]
    T:       11.4314482 +/- 3.44267676 (30.12%) (init = 20)
    lnNugu:  21.8986958 +/- 0.50305128 (2.30%) (init = 20)
[[Correlations]] (unreported correlations are < 0.100)
    C(T, lnNugu) = -0.951


### (Multiple components)

In [15]:
from lmfit import minimize, Parameters, fit_report


def residual(params, x, data, eps_data):
    m = -1/params['T']
    b = params['lnNugu']
  
    model = m*x+b

    return (data-model) / eps_data


params = Parameters()
params.add('T', value=20, min=0, max=1000)
params.add('lnNugu', value=20)

out_low = minimize(residual, params, args=(np.array(filtered_energ_low), filtered_lognugu_low, filtered_err_low))

params = Parameters()
params.add('T', value=20, min=0, max=1000)
params.add('lnNugu', value=20)

out_high = minimize(residual, params, args=(np.array(filtered_energ_high), filtered_lognugu_high, filtered_err_high))

print(fit_report(out_low))
print("--------------------------------------------------------------------")

print(fit_report(out_high))

# <end examples/doc_fitting_withreport.py>

[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 33
    # data points      = 6
    # variables        = 2
    chi-square         = 258.345896
    reduced chi-square = 64.5864741
    Akaike info crit   = 26.5752394
    Bayesian info crit = 26.1587583
[[Variables]]
    T:       999.997972 +/- 628213.870 (62821.51%) (init = 20)
    lnNugu:  23.7907877 +/- 20.3634681 (85.59%) (init = 20)
[[Correlations]] (unreported correlations are < 0.100)
    C(T, lnNugu) = -0.998
--------------------------------------------------------------------
[[Fit Statistics]]
    # fitting method   = leastsq
    # function evals   = 32
    # data points      = 6
    # variables        = 2
    chi-square         = 143.362653
    reduced chi-square = 35.8406632
    Akaike info crit   = 23.0417079
    Bayesian info crit = 22.6252268
[[Variables]]
    T:       999.999999 +/- 41063.7277 (4106.37%) (init = 20)
    lnNugu:  25.7900500 +/- 5.61261942 (21.76%) (init = 20)
[[Correlations]] (unr

# Single component analysis

In [66]:
T = out.params["T"]
lnNugu = out.params["lnNugu"]

Eu_linspace = np.linspace(min(filtered_energ), max(filtered_energ), 1000)

def rotdiag_model(Eu, T, lnNugu):
    return lnNugu - (1/T)*Eu

plt.errorbar(energies, lognugu, yerr=errors, fmt="o", capsize=3, color="gray")
plt.errorbar(filtered_energ, filtered_lognugu, yerr=filtered_err, fmt="o", capsize=3, color="red")

plt.plot(Eu_linspace, rotdiag_model(Eu_linspace, T, lnNugu), color="red", linestyle="--")
    
plt.ylabel("$\ln(N_u/g_u)$", fontsize=15)
plt.xlabel("$E_u/k$", fontsize=15)
plt.show()

### Derive total column density $N_{tot}$ by approximating the rotational partition function $Q_{rot}$

In [67]:
# CH3CN
Qrot = [13.8355, 28.4924, 64.0955, 164.3168, 449.0811, 1267.6705, 2628.0493]
Trot = [2.725, 5.0, 9.375, 18.75, 37.5, 75.0, 120.0]

# Values to interpolate from

# CH3OH
#Qrot = [11.889916, 26.719018, 78.173628, 274.987967, 920.963739, 2924.302297, 9750.039754]
#Trot = [2.725, 5.0, 9.375, 18.75, 37.5, 75.0, 150.0]

# CH3OCH3
#Qrot = [2669.7988, 7495.701, 21137.3351, 61604.1674, 221990.1994, 545256.5114, 1060153.51]
#Trot = [9.375, 18.75, 37.5, 75.0, 150.0, 225.0, 300.0]

# CH2CO
#Qrot = [47.577, 157.217,  457.088,  1285.879, 3433.998]
#Trot = [9.375, 18.75, 37.50, 75.00, 150.0]

# HC3N
#Qrot = [43.2767, 86.2186, 172.1063, 343.8929, 687.5086]
#Trot = [9.375, 18.75, 37.5, 75.0, 150.0]

# CH3CCH
# Qrot = [68.7664, 176.2982, 481.7197, 1362.3596, 4209.3392]
# Trot = [9.375, 18.75, 37.5, 75.0, 150.0]

# H2SO4
#Qrot = [1, 865.8732, 2441.3042, 6894.5189,  19488.0808, 55115.9292]
#Trot = [0, 9.375, 18.75, 37.5, 75.0, 150.0]

# C4H
#Qrot = [165.5392,  329.7433,  658.1711, 1315.0879]
#Trot = [9.375, 18.75 , 37.50, 75.00]

# C6H
# Qrot = [626.7718, 1487.9395, 3519.7042, 7871.6218]
# Trot = [9.375, 18.75, 37.50, 75.00]

# C2H3CN
#Qrot = [437.499, 1232.852, 3481.779, 9958.401]
#Trot = [9.375, 18.75, 37.50, 75.00]

Qfit = np.polyfit(Trot, Qrot, 3)
Q = np.poly1d(Qfit)

T_linspace = np.linspace(min(Trot), max(Trot))
plt.scatter(Trot, Qrot, c="black")
plt.plot(T_linspace, Q(T_linspace), color="black")

Qerr = np.sqrt((T*T.stderr)*(3*(Qfit[0]**2)*(T**4)+2*Qfit[1]*T**2+Qfit[1]**2))

plt.errorbar([T], [Q(T)], yerr=Qerr , c="red", fmt="*", capsize=3)

plt.ylabel("$Q_{rot}$", fontsize=15)
plt.xlabel("$T$", fontsize=15)
plt.show()

In [68]:
lnNtot = lnNugu + np.log(Q(T))
lnNtot_err = np.sqrt((lnNugu.stderr)**2 + (Qerr/Q(T))**2)

logNtot = np.log10(np.exp(lnNtot))
logNtot_err = 0.434*(np.exp(lnNtot)*lnNtot_err/np.exp(lnNtot))

rot_err = np.sqrt((lnNugu.stderr)**2 + (np.array(Eu_linspace)*T**(-2)*T.stderr)**2)

In [69]:
plt.subplot2grid((1,7), (0,0), colspan=5)

#plt.errorbar(energies, lognugu, yerr=errors, fmt="o", capsize=3, color="gray")
plt.errorbar(filtered_energ, filtered_lognugu, yerr=filtered_err, fmt="o", capsize=3, color="red")

plt.plot(Eu_linspace, rotdiag_model(Eu_linspace, T, lnNugu), color="red", linestyle="--")
    
plt.ylabel("$\ln(N_u/g_u)$", fontsize=20)
plt.xlabel("$E_u/k$", fontsize=20)


plt.fill_between(x=Eu_linspace, y1=rotdiag_model(Eu_linspace, T, lnNugu)-rot_err, y2=rotdiag_model(Eu_linspace, T, lnNugu)+rot_err, alpha=0.1, color="gray")

plt.subplot2grid((1,7), (0,5), colspan=1)
plt.errorbar([0], [logNtot], yerr=logNtot_err , c="black", capsize=3)

plt.fill_between(x=[-1,1],y1=0,y2=logNtot, color="lime", alpha=0.6)
plt.title("$\log_{10}(N_{tot})$", fontsize=20)

plt.xticks([])
plt.ylim([0,20])

plt.subplot2grid((1,7), (0,6), colspan=1)
plt.errorbar([0], [T], yerr=T.stderr , c="black", capsize=3)

plt.fill_between(x=[-1,1],y1=0,y2=T, color="lime", alpha=0.6)
plt.title("$T_{rot}$(K)", fontsize=20)

plt.xticks([])
plt.ylim([0,50])

plt.show()

## Multiple component analysis

In [17]:
T_low = out_low.params["T"]
lnNugu_low = out_low.params["lnNugu"]

Eu_linspace_low = np.linspace(min(filtered_energ_low)-5, max(filtered_energ_low)+5, 1000)

def rotdiag_model(Eu, T, lnNugu):
    return lnNugu - (1/T)*Eu

#plt.errorbar(energies, lognugu, yerr=errors, fmt="o", capsize=3, color="gray")
plt.errorbar(filtered_energ_low, filtered_lognugu_low, yerr=filtered_err_low, fmt="o", capsize=3, color="blue")

plt.plot(Eu_linspace_low, rotdiag_model(Eu_linspace_low, T_low, lnNugu_low), color="blue", linestyle="--")

Eu_linspace_high = np.linspace(min(filtered_energ_high)-5, max(filtered_energ_high)+5, 1000)

T_high = out_high.params["T"]
lnNugu_high = out_high.params["lnNugu"]

plt.errorbar(filtered_energ_high, filtered_lognugu_high, yerr=filtered_err_high, fmt="o", capsize=3, color="red")

plt.plot(Eu_linspace_high, rotdiag_model(Eu_linspace_high, T_high, lnNugu_high), color="red", linestyle="--")

plt.ylabel("$\ln(N_u/g_u)$", fontsize=15)
plt.xlabel("$E_u/k$", fontsize=15)
plt.show()

In [ ]:
# H2CO
Qrot = [13.801, 44.681, 128.649, 361.720, 1019.971, 1872.622, 2883.014]
Trot = [9.375, 18.75, 37.50, 75.00, 150.0, 225.0, 300.0]


Qfit = np.polyfit(Trot, Qrot, 3)
Q = np.poly1d(Qfit)

T_linspace = np.linspace(min(Trot), max(Trot))
plt.scatter(Trot, Qrot, c="black")
plt.plot(T_linspace, Q(T_linspace), color="black")

Qerr_low = np.sqrt((T_low*T_low.stderr)*(3*(Qfit[0]**2)*(T_low**4)+2*Qfit[1]*T_low**2+Qfit[1]**2))
Qerr_high = np.sqrt((T_high*T_high.stderr)*(3*(Qfit[0]**2)*(T_high**4)+2*Qfit[1]*T_high**2+Qfit[1]**2))

plt.errorbar([T_high], [Q(T_high)], yerr=Qerr_high , c="red", fmt="*", capsize=3)
plt.errorbar([T_low], [Q(T_low)], yerr=Qerr_low , c="blue", fmt="*", capsize=3)

plt.ylabel("$Q_{rot}$", fontsize=15)
plt.xlabel("$T$", fontsize=15)
plt.show()

In [ ]:
lnNtot_low = lnNugu_low + np.log(Q(T_low))
lnNtot_err_low = np.sqrt((lnNugu_low.stderr)**2 + (Qerr_low/Q(T_low))**2)

logNtot_low = np.log10(np.exp(lnNtot_low))
logNtot_err_low = 0.434*(np.exp(lnNtot_low)*lnNtot_err_low/np.exp(lnNtot_low))

rot_err_low = np.sqrt((lnNugu_low.stderr)**2 + (np.array(Eu_linspace_low)*T_low**(-2)*T_low.stderr)**2)
#''''''
lnNtot_high = lnNugu_high + np.log(Q(T_high))
lnNtot_err_high = np.sqrt((lnNugu_high.stderr)**2 + (Qerr_high/Q(T_high))**2)

logNtot_high = np.log10(np.exp(lnNtot_high))
logNtot_err_high = 0.434*(np.exp(lnNtot_high)*lnNtot_err_high/np.exp(lnNtot_high))

rot_err_high = np.sqrt((lnNugu_high.stderr)**2 + (np.array(Eu_linspace_high)*T_high**(-2)*T_high.stderr)**2)

In [ ]:
plt.subplot2grid((1,7), (0,0), colspan=5)

#plt.errorbar(energies, lognugu, yerr=errors, fmt="o", capsize=3, color="gray")
plt.errorbar(filtered_energ_low, filtered_lognugu_low, yerr=filtered_err_low, fmt="o", capsize=3, color="blue")
plt.plot(Eu_linspace_low, rotdiag_model(Eu_linspace_low, T_low, lnNugu_low), color="blue", linestyle="--") 
plt.ylabel("$\ln(N_u/g_u)$", fontsize=20)
plt.xlabel("$E_u/k$", fontsize=20)
plt.fill_between(x=Eu_linspace_low, y1=rotdiag_model(Eu_linspace_low, T_low, lnNugu_low)-rot_err_low, y2=rotdiag_model(Eu_linspace_low, T_low, lnNugu_low)+rot_err_low, alpha=0.1, color="gray")

plt.errorbar(filtered_energ_high, filtered_lognugu_high, yerr=filtered_err_high, fmt="o", capsize=3, color="red")

plt.plot(Eu_linspace_high, rotdiag_model(Eu_linspace_high, T_high, lnNugu_high), color="red", linestyle="--")
    
plt.ylabel("$\ln(N_u/g_u)$", fontsize=20)
plt.xlabel("$E_u/k$", fontsize=20)


plt.fill_between(x=Eu_linspace_high, y1=rotdiag_model(Eu_linspace_high, T_high, lnNugu_high)-rot_err_high, y2=rotdiag_model(Eu_linspace_high, T_high, lnNugu_high)+rot_err_high, alpha=0.1, color="gray")

plt.subplot2grid((1,7), (0,5), colspan=1)
plt.errorbar([-0.5], [logNtot_low], yerr=logNtot_err_low , c="black", capsize=3)
plt.errorbar([0.5], [logNtot_high], yerr=logNtot_err_high , c="black", capsize=3)

plt.fill_between(x=[-1,0],y1=0,y2=logNtot_low, color="blue", alpha=0.6)
plt.fill_between(x=[0,1],y1=0,y2=logNtot_high, color="red", alpha=0.6)


plt.title("$\log_{10}(N_{tot})$", fontsize=20)

plt.xticks([])
plt.ylim([0,20])

plt.subplot2grid((1,7), (0,6), colspan=1)
plt.errorbar([-0.5], [T_low], yerr=([[T_low], [T_low.stderr]]) , c="black", capsize=3)
plt.errorbar([0.5], [T_high], yerr=T_high.stderr , c="black", capsize=3)

plt.fill_between(x=[-1,0],y1=0,y2=T_low, color="blue", alpha=0.6)
plt.fill_between(x=[0,1],y1=0,y2=T_high, color="red", alpha=0.6)

plt.title("$T_{rot}$(K)", fontsize=20)

plt.xticks([])
plt.ylim([0,200])

plt.show()
